# BRAZ Lucas & DURAND Pierre-Alain
## SCIPER: 343141 & SCIPER: 344313

In [1]:
import numpy as np
import random

from tic_env import TictactoeEnv, OptimalPlayer

import plotly.express as px

In [2]:
def play_games(player_opt, agent, maxGames=20_000, env=TictactoeEnv()):
    Turns = np.array(['X','O'])
    winnerList = np.zeros(maxGames)
    for nbGames in range(maxGames):
        env.reset()
        grid, _, __ = env.observe()

        player_opt.player = Turns[nbGames%2]
        agent.player = Turns[(nbGames+1)%2]

        for roundGame in range(9):
            if env.current_player == player_opt.player:
                move = player_opt.act(grid)
            else:
                if roundGame > 1:
                    agent.learn(grid, 0)
                move = agent.act(grid)            

            grid, end, winner = env.step(move, print_grid=False)

            if end:
                if winner == agent.player:
                    winnerList[nbGames] = 1
                    agent.learn(grid, 1, end=True)
                elif winner == player_opt.player:
                    winnerList[nbGames] = -1
                    agent.learn(grid, -1, end=True)
                else:
                    agent.learn(grid, 0, end=True)
                break
    env.reset()
    return winnerList

# 2 *Q*-Learning

In [3]:
class Qtable:
    def __init__(self, Qtab={}, defaultValue=0.0):
        self.Qtab = Qtab
        self.defaultValue = defaultValue

    def hash(self, state_action):
        """
        hash a state-action tuple to a unique integer. Each element of the grid (state) has 3 states, 
        same for the action position.
        Then, we consider the state-action pair as a base 3 number, and we hash it to an integer.
        Can be easily recover from hash to state-action pair.
        """
        state, action = state_action

        hashState = (state.flatten()+1)@np.array([3**0,3**1,3**2,3**3,3**4,3**5,3**6,3**7,3**8])
        hashAction = action[0]*3**9 + action[1]*3**10
        return hashState + hashAction
    
    def __getitem__(self, key: tuple):
        return self.Qtab.get(self.hash(key), self.defaultValue)

    def __setitem__(self, key: tuple, value):
        self.Qtab[self.hash(key)] = value

In [4]:
class QlearningAgent:
    """
    Description:
        A class to implement an epsilon-greedy Qlearning Agent in Tic-tac-toe.

    Parameters:
        epsilon: float, in [0, 1]. This is a value between 0-1 that indicates the
            probability of making a random action instead of the greedy action
            at any given time.

        learningRate: float, in [0, 1]. Setting it to 0 means that the Q-values are
            never updated, hence nothing is learned. Setting a high value such as 0.9 
            means that learning can occur quickly. 
        
        discountFactor: float, in [0, 1]. This models the fact that future rewards are
            worth less than immediate rewards. Setting it to 0 means that the agent
            will only learn from immediate rewards. Setting it to 1 means that the
            agent will learn from all rewards equally.
    """

    def __init__(self, epsilon=0.2, player='X', learningRate=0.05, discountFactor=0.99, Q={}, Q_defaultValue=0.0, n_max=100):
        if isinstance(epsilon, tuple):
            self.epsilon_min, self.epsilon_max = epsilon
            self.epsilon = self.epsilon_max
        else:
            self.epsilon = epsilon
            self.epsilon_min = epsilon
            self.epsilon_max = epsilon
        self.learningRate = learningRate
        self.discountFactor = discountFactor

        self.Q = Qtable(Q, Q_defaultValue)
        self.state = None
        self.action = None

        self.n = 0
        self.n_max = n_max

        self.isLearning = True

        self.player = player # 'X' or 'O'

    def decrease_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon_max * (1 - self.n / self.n_max))


    def set_player(self, player = 'X', j=-1):
        self.player = player
        if j != -1:
            self.player = 'X' if j % 2 == 0 else 'O'

    def empty(self, state):
        """ Return all empty positions. """
        availableActions = []
        for x in range(3):
            for y in range(3):
                position = (x, y)
                if state[position] == 0:
                    availableActions.append(position)
        return availableActions

    def randomAction(self, state):
        """ Choose a random action from the available options. """
        availableActions = self.empty(state)

        return random.choice(availableActions)

    def bestAction(self, state):
        """
        Choose the available actions which have a maximum expected future reward. 
        If there are multiple actions with the same maximum expected future reward,
        choose one of them at random.
        """
        # Get the available moves
        availableActions = self.empty(state)

        # Get the best move
        bestActions = []
        bestValue = -999.0
        for action in availableActions:
            Qsa = self.Q[(state, action)]
            if Qsa == bestValue:
                bestActions.append(action)
            if Qsa > bestValue:
                bestActions = [action]
                bestValue = Qsa

        return random.choice(bestActions)

    def act(self, state):
        """
        epsilon-greedy action selection, according to the Q-table.
        """
        self.state = state

        # whether move in random or not
        if random.random() < self.epsilon:
            self.action = self.randomAction(state)
        else:
            # Get the best move
            self.action = self.bestAction(state)

        return self.action

    def learn(self, s_prime, reward, end=False):
        """
        Q-learning update. If it's the end of a game, we set Q(s',a') = 0.
        """
        if self.isLearning:
            if not end:
                # Get the best move
                a_prime = self.bestAction(s_prime)

                # Update the Q-value
                self.Q[(self.state, self.action)] += self.learningRate * (reward + self.discountFactor * self.Q[(s_prime, a_prime)] - self.Q[(self.state, self.action)])
            else:
                self.Q[(self.state, self.action)] += self.learningRate * (reward - self.Q[(self.state, self.action)])

                self.state = None
                self.action = None

                self.n += 1
                self.decrease_epsilon()

## 2.1 Learning from experts

#### Question 1

In [5]:
epsilon = 0.4

player_opt = OptimalPlayer(epsilon=0.5)
agent = QlearningAgent(epsilon=epsilon)

winnerList = play_games(player_opt, agent, maxGames=20_000)

x = np.arange(winnerList.size//50-1)
y = np.zeros(x.shape)
for i in x:
    y[i] = winnerList[i*50:(i+1)*50].mean()

fig = px.line(x=50*x, y=y, title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.update_layout(xaxis_title='Game number', yaxis_title='Average reward')
fig.show()

### 2.1.1 Decreasing exploration

#### Question 2

In [25]:
epsilon = (0.1, 0.8)
n_max = 500

player_opt = OptimalPlayer(epsilon=0.5)
agent = QlearningAgent(epsilon=epsilon, n_max=n_max)

winnerList = play_games(player_opt, agent, maxGames=20_000)

groupSize = 250
x = np.arange(winnerList.size//groupSize-1)
y = np.zeros(x.shape)
for i in x:
    y[i] = winnerList[i*groupSize:(i+1)*groupSize].mean()

fig = px.line(x=groupSize*x, y=y, title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.update_layout(xaxis_title='Game number', yaxis_title='Average reward')
fig.show()

.